In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline


tf_models_path  = '/home/ubuntu/projects/training/tensorflow/models/research/'
images_path = '/home/ubuntu/data/training/image/face_detection_dataset/test/images'

# Select GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"


In [ ]:
# Import util functions from the Object Detection API

sys.path.append(tf_models_path)
sys.path.append(tf_models_path + "object_detection")

from utils import label_map_util
from utils import visualization_utils as vis_util


In [ ]:
# Model to load

PATH_TO_CKPT = tf_models_path + 'object_detection/models/faces/train/output_inference_graph/frozen_inference_graph.pb'
#PATH_TO_MODEL =  tf_models_path + 'object_detection/models/faces/train/model.ckpt-66887'
PATH_TO_LABELS = tf_models_path + 'object_detection/data/faces/faces_label_map.pbtxt'
NUM_CLASSES = 1


In [ ]:
# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [ ]:
# Load model
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


In [ ]:

TEST_IMAGE_PATHS = [ os.path.join(images_path, '00{}.jpg'.format(i)) for i in range(33, 45) ]
print('Num test images to check:', len(TEST_IMAGE_PATHS))


In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
            
            # Read image
            image = Image.open(image_path)
            
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            # Execute inference.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=4)
            plt.figure(figsize=(20, 14)) # Size, in inches, of the output images
            plt.imshow(image_np)

In [ ]:
%env